In [1]:
from kfp import dsl

In [ ]:
@dsl.component(packages_to_install=['numpy','pandas','joblib','scikit-learn','flask'])
def modeldevelopment(rooms: int,sqft: int) -> float:
        # Importing Libraries
        import pandas as pd
        import numpy as np
        import joblib
        from flask import Flask, render_template, request
        from sklearn.linear_model import LinearRegression
        from sklearn.model_selection import train_test_split
        from sklearn.metrics import mean_squared_error
        
        # Load and process the data
        # rentalDF = pd.read_csv('data/rental_1000.csv')
        rentalDF = pd.read_csv('https://raw.githubusercontent.com/azonecloud/0800amist-learning-mlops/main/mlops-predict-rental-price/data/rental_1000.csv')
        
        # Data Transformation (Feature Engineering - Use Features for Model Development)
        X = rentalDF[['rooms', 'sqft']].values   # Features 
        y = rentalDF['price'].values             # Label
        
        # Split Data into Training and Testing 
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
        
        # Model Training
        model = LinearRegression().fit(X_train, y_train)

        # Example prediction for a specific test sample
        #sample_index = 0  # Change this index to test different samples
        #predict_rental_price = model.predict([X_test[sample_index]])[0]
        #print(f"The Real Rental Price for Rooms count={X_test[sample_index][0]} and Area in Sqft={X_test[sample_index][1]} is={y_test[sample_index]}")
        #print(f"The Predicted Rental Price for Rooms count={X_test[sample_index][0]} and Area in Sqft={X_test[sample_index][1]} is={predict_rental_price}")

        predict_rental_price = model.predict(np.array([[rooms,sqft]]))[0]
        print(f"The Predicted Rental Price for Rooms is={predict_rental_price}")

        return (predict_rental_price)

        # Save the Model
        #joblib.dump(model, 'model/rental_price_model.joblib')
        
        # Load the Model
        #model = joblib.load('model/rental_price_model.joblib')
        
        # Initialize Flask application
        #app = Flask(__name__)
        
        #@app.route('/')
        #def home():
            #return render_template('index.html')
        
        #@app.route('/predict', methods=['POST'])
        #def predict():
            #rooms = int(request.form['rooms'])
            #sqft = int(request.form['sqft'])
        
            # Make the prediction
            #prediction = model.predict(np.array([[rooms, sqft]]))
            
            #return render_template('result.html', prediction=prediction[0])
        
        if __name__ == "__main__":
            app.run(debug=True, host="0.0.0.0")
        
        # Compute RMSE (optional)
        # y_pred = model.predict(X_test)
        # rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        # print(f"RMSE: {rmse}")
        
        # Compute RMSE
        # rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        # print(f"RMSE: {rmse}")

In [3]:
@dsl.pipeline
def rental_prediction_pipeline(rooms_count:int,area_in_sqft:int) -> float:
    task = modeldevelopment(rooms=rooms_count,sqft=area_in_sqft)
    return task.output

In [4]:
if __name__ == "__main__":
    # Compile the pipeline
    #compiler.Compiler().compile(rental_prediction_pipeline, 'rental_prediction_pipeline.yaml')

    # Connect to Kubeflow Pipelines server
    kfp_endpoint = None  # Replace with the actual URL of your Kubeflow Pipelines server
    client = kfp.Client(host=kfp_endpoint)

    # Create an experiment
    experiment_name = 'Experiment for Model to Predict Rental Price'
    experiment = client.create_experiment(name=experiment_name)
    print(f'Experiment created: {experiment}')

    # Run the pipeline using create_run_from_pipeline_func
    run_name = 'Run of Rental Price Prediction Pipeline'
    run_result = client.create_run_from_pipeline_func(
        pipeline_func=rental_prediction_pipeline,
        run_name=run_name,
        experiment_name=experiment_name,
        arguments={'rooms_count': 3, 'area_in_sqft': 1500}  # Provide the actual parameters here
    )

    print(f'Pipeline run submitted: {run_result}')